In [1]:
'''*********************************************** primal *******************************************************'''
from file import *

def primal():
    problem_type = 'min'
    objective_function0 = [5, 12, 7, 18]
    

    restrictions = [[1, 0, 3, -3],
                    [0, 2, 2, -2]
                   ]
                  
    
    operators = ['>=',
                 '<=']
            
    b0= [3, 5]

    ############################## Dual entrants ###############################
    d_operators=[]
    if problem_type=='max':
        d_problem_type='min'
        rest=np.array(restrictions)
        dual_rest=rest.T
        d_restrictions=dual_rest.tolist()
        d_b=objective_function0
        d_objective_function0=b0
        
        for i in range(len(d_b)):
            d_operators.append('>=')
    else:
        d_problem_type='max'
        rest=np.array(restrictions)
        dual_rest=rest.T
        d_restrictions=dual_rest.tolist()
        d_b=objective_function0
        d_objective_function0=b0
        for i in range(len(d_b)):
            d_operators.append('<=')
    
   
    ############################################################################
    obj=[]
    for i in range(len(restrictions[0])):
        obj.append(objective_function0[i])
    #########################non_unique optimal solution#######################
    
    s=[]
    for i in range(len(obj)):
        s.append(0)
    f=[]
    bb=500000000000
    mat_objective_function0=np.array(objective_function0)
    mat_rest=np.array(restrictions)

    for i in range(len(b0)):
        f.append(mat_objective_function0/mat_rest[i])
    #print('$$$$$$$$$$$$',f)
    for i in range(len(b0)):
        if len(np.unique(f[i]))==1:
            s=restrictions[i]
            bb=b0[i]
            rest_index=i+1
    s=np.array(s)
    s=s.reshape(1,len(obj))
    ###########################################################################
    # creating standard form #
    A, objective_function, b, basic_vars, Non_basic_vars, fase1, nVarArtificial = standard_form(problem_type, objective_function0, restrictions, operators, b0)

    problem = False

    if fase1:
        itr = 1
        objective_function_fase1 = [0] * len(objective_function)
        for i in range(len(objective_function_fase1) - 1, len(objective_function_fase1) - 1 - nVarArtificial, -1): # assigns 1 for all costs of artificial variables
            objective_function_fase1[i] = 1

        print('\n\n\033[33mFase I\033[m')
        print('Objective function of fase I: {}'.format(objective_function_fase1))
      

        while True:

            B = update_B(A, basic_vars)
            B_inv=np.linalg.inv(B)
            
            CB=[]
            for i in basic_vars:
                CB.append(objective_function_fase1[i])
            
            printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
            Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))
            mXb=np.array(Xb)
            lbda = step2_1(B, basic_vars, objective_function_fase1)
            #SN = CN - NT* lamda
            vet_cnk = step2_2(objective_function_fase1, lbda, A, Non_basic_vars)
            ##########################################################
            Binvdotrest=np.dot(B_inv, restrictions)
            
            CBdot=np.dot(CB,Binvdotrest)
#             print('CB*B^-1*A:',CBdot)
#             print("\n")
           
            CBXb=np.dot(CB,Xb)
#             print('CB*Xb :',CBXb)
#             print("\n")
            CBBinv=np.dot(CB,B_inv)
#             print('CB*B^-1 :',CBBinv)
#             print("\n")
            BinvA=np.dot(B_inv,restrictions)
#             print('B^-1*A :', BinvA)
#             print("\n")
           
            
            ##
            finish, k = step3(vet_cnk, True)

            if finish:
                break
            # column of the intery variable in table 
            y = step4(B, A, Non_basic_vars, k)
            #finding the exiting variable by considering the ratios(index of B column)
            leaver, problem = step5_fase1(Xb, y)
#         ########################checking degeneracy#######################
         
            igreg=np.array(y)
            igreg=igreg.reshape(len(y),1)
            mXb=mXb.reshape((len(Xb)),1)
            #print('dddddddddddddddd',np.shape(mXb),np.shape(igreg))
            ratio=mXb/igreg
            #print('ffffffffff',mXb,y,ratio)
            
            ratio=np.reshape(ratio,len(y))
            ratio_list=ratio.tolist()
            t1 = min(i for i in ratio_list if i >= 0)
            ratio_list.remove(t1)
            a1 = [x for x in ratio_list if x>=0]
            if len(a1)>0 :
              t2 = min(a1)
              if t1 == t2 :
                print(colored('we have degeneracy!','red'))

        
        
#         ##################################################################

            if problem:
                break

            basic_vars, Non_basic_vars, finish = step6_fase1(basic_vars, Non_basic_vars, k, leaver, nVarArtificial)

            if finish:
                break

            itr += 1

        fimFase1(A, nVarArtificial, Non_basic_vars)

    itr = 1
    X = []

    print('\n\n\033[33mFase II\033[m')
    while True:
        

        B = update_B(A, basic_vars)
        
        B_inv=np.linalg.inv(B)
        mB_inv=np.array(B_inv)
        #
        mB_inv=np.around(mB_inv, decimals=5)
        #
        CB=[]
        for i in basic_vars:
            
            CB.append(objective_function[i])
        mCB=np.array(CB)
        printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
        Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))
        

        row_titles=['z']
        for i in basic_vars:
            row_titles.append('x'+str(i+1))

        mrow_titles=np.array(row_titles)
        reshaped_row_titles=mrow_titles.reshape(len(row_titles),1)

        lbda = step2_1(B, basic_vars, objective_function)
        vet_cnk = step2_2(objective_function, lbda, A, Non_basic_vars)
        
        
        ############################################################
        len_mXb=len(Xb)
        mXb=np.array(Xb)
        #
        mXb=np.around(mXb, decimals=5)
        #

        Binvdotrest=np.dot(B_inv, restrictions)
        mBinvdotrest=np.array(Binvdotrest)
 
        CBBinvA=np.dot(CB,Binvdotrest)
        mCBBinvA=np.array(CBBinvA)
#         print('CB*B^-1*A:',CBBinvA)
#         print("\n")
        if problem_type == 'min':
            CBBinvA_c=CBBinvA-obj
        else:
            CBBinvA_c=CBBinvA+obj
        mCBBinvA_c=np.array(CBBinvA_c)
        #
        mCBBinvA_c=np.around(mCBBinvA_c, decimals=5)
        #
#         print('(CB*B^-1*A)-c :',mCBBinvA_c)
#         print("\n")
        
        CBXb=np.dot(CB,Xb)
        mCBXb=np.array(CBXb)
        #
        mCBXb=np.around(mCBXb, decimals=5)
        #
        
#         print('CB*Xb :',CBXb
        CBBinv=np.dot(CB,B_inv)
        mCBBinv=np.array(CBBinv)
        #
        mCBBinv=np.around(mCBBinv, decimals=5)
        #
#         print('CB*B^-1 :',CBBinv)
#         print("\n")
        BinvA=np.dot(B_inv,restrictions)
        mBinvA=np.array(BinvA)
        #
        mBinvA=np.around(mBinvA, decimals=5)
        #
#         print('B^-1*A :', BinvA)
#         print("\n")
        BinvABinv=np.concatenate((mBinvA,mB_inv),1)
   
        mXb=mXb.reshape((len_mXb,1))
        BinvABinvXb=np.concatenate((BinvABinv,mXb),1)
#         print('B^-1*A/B^-1/B^-1*b :',BinvABinvXb)
#         print("\n")

        CBBinvA_cCBBinv=np.concatenate((mCBBinvA_c,mCBBinv),None)

        row0=np.concatenate((CBBinvA_cCBBinv, mCBXb),None)
       
#         print('CB*B^-1*A-c/CB*B^-1/CB*B^-1*b:', row0)
#         print("\n")
        len_row0=len(row0.tolist())
        
        table=np.concatenate((row0.reshape((1,len_row0)),BinvABinvXb),0)
        table=np.concatenate((reshaped_row_titles,table),1)
       ############################################################################ 

        #################### fixing table for '=' operators###############
        
        
        delete=[]
        for i in range(len(operators)):
            if operators[i] == '=':
                delete.append(i)

        a=0    
        delete=sorted(delete)
    #if 
        for i in range(len(delete)):

            table = np.delete(table,len(obj)+delete[i]+a+1, 1)
            a += -1
            #print('nt',table)
        
        ############################## table ####################################
        p,u=np.shape(table)
        #print(p,u)
        x = PrettyTable()
        column_num=['_']
        for j in range(u-1):
            column_num.append('X'+str(j+1))
        column_num[j+1]='RHS'
        for i in range(p):
            x.add_row(table[i,:])

        x.field_names = column_num
        print(x)
        ####################################################################
                              
        finish, k = step3(vet_cnk, False)

        if finish:
            break

        y = step4(B, A, Non_basic_vars, k)
        leaver, problem = step5(Xb, y)
        
#         ########################checking degeneracy#######################
        
        igreg2=np.array(y)
        igreg2=igreg2.reshape(len(y),1)
        mXb=mXb.reshape((len(Xb)),1)
        ratio2=mXb/igreg2
        ratio2=np.reshape(ratio2,len(y))
        ratio_list2=ratio2.tolist()
        t12 = min(i for i in ratio_list2 if i >= 0)
        ratio_list2.remove(t12)
        a12 = [x for x in ratio_list if x>=0]
        if len(a12)>0 :
          t22 = min(a12)
          if t12 == t22 :
            print(colored('we have degeneracy!','red'))

#         ##################################################################
        

        if problem:
            break

        basic_vars, Non_basic_vars = step6(basic_vars, Non_basic_vars, k, leaver)
        
        itr += 1

    if not problem:
        e=[]
        for i in range(len(restrictions[0])):
            e.append(X[i])
        e=np.array(e)
        e=e.reshape(len(obj),1)
        dot=np.dot(s,e)

        print('\033[33mFinal Primal Table\033[m')
        
        final(restrictions, objective_function, problem_type, X)
        
        if dot[0]==bb:
            print(colored('we have none_unique optimal solution !','red'))
            print(colored('All the points on restriction'+str(rest_index)+' are optimal solutions.' ,'red'))
            print("\n\n")   
        ################## sensitivity analysis 1 ####################
        print("\n\n")
        import sympy as sym
        from sympy.solvers.inequalities import reduce_rational_inequalities
        for j in range(len(b0)):
            cc = sym.Symbol('b'+str(j)) 
            testb=deepcopy(b0)
            testb[j]=cc
            testb=np.array(testb)
            testb=testb.reshape(len(b0),1)
            
            BinvXb=np.dot(mB_inv,testb)
            BinvXb=np.reshape(BinvXb,len(b0))
            
            print(colored('sensitivity analysis b'+str(j)+' :','green'))
            for i in range(len(b0)):
            
                print(reduce_rational_inequalities([[BinvXb[i] > 0]], cc))

        ################## sensitivity analysis 2 #######################
#         print("\n\n")
 
#         for f in range(len(obj)):
#             bv=deepcopy(basic_vars)
#             bv=np.array(bv)
#             AA=np.array(restrictions)
#             AA=AA.reshape(len(b0),len(obj))
#             uu=sym.Symbol('c'+str(f))
#             ob=deepcopy(obj)
#             ob[f]=uu
#             result=[]
#             Cb=deepcopy(CB)
#             result=np.where(bv == f)
#             result=np.array(result)
#             Cb[result[0][0]]= uu
# #             print('kkkkkkkkk',result,result[0][0],Cb,ob)
# #             print('jjjjjjjjjjjjjjj',mB_inv)
#             newCB=np.array(Cb)
#             newCB=newCB.reshape(1,len(Cb))
#             CBBinverse=np.dot(newCB,mB_inv) #
#             CBBinverseA=np.dot(CBBinverse,AA)
#             CBBinverse=np.reshape(CBBinverse,len(CB))
#             CBBinverseA_c=CBBinverseA-ob #
#             CBBinverseA_c=np.reshape(CBBinverseA_c,len(obj))
#             print(colored('sensitivity analysis c'+str(f)+' :','blue'))
#            # print('ffffffffffffffff',CBBinverseA_c,CBBinverse)
#             for j in range(len(b0)):
#                 print(reduce_rational_inequalities([[ CBBinverse[j] >= 0]], uu ))
#             for h in range(len(obj)):
#                 print(reduce_rational_inequalities([[ CBBinverseA_c[h] >= 0]], uu ))
                

        ############################### dual entrants ##################################  
    if problem_type == 'max':
      for i in range(len(operators)):
        if operators[i] == '>=':
          d_objective_function0[i] *= -1
          d_restrictions = np.array(d_restrictions)
          d_restrictions[:,i] *= -1
          d_restrictions = d_restrictions.tolist()
    if problem_type == 'min':
      for i in range(len(operators)):
        if operators[i] == '<=':
          d_objective_function0[i] *= -1
          d_restrictions = np.array(d_restrictions)
          d_restrictions[:,i] *= -1
          d_restrictions = d_restrictions.tolist()
    return(d_problem_type,d_restrictions,d_objective_function0,d_operators,obj,b0)


d_problem_type,d_restrictions,d_objective_function0,d_operators,d_b,b0 =primal()
print("\n\n\n")
print('d_problem_type:',d_problem_type,'--d_restrictions:',d_restrictions,'--d_objective_function0:',d_objective_function0,'--d_operators:',d_operators,'--d_b:',d_b,'--b0:',b0)


/home/shiny/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in true_divide
/home/shiny/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:130: RuntimeWarning: divide by zero encountered in true_divide
/home/shiny/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:303: RuntimeWarning: divide by zero encountered in true_divide



Matrix A:
1	0	3	-3	-1	0	1	0	
0	2	2	-2	0	1	0	1	


Objective function: [5, 12, 7, 18, 0, 0, 0, 0]
Vector b: [3, 5]
Index of Basic variables: [6, 7]
Index of Non-basic variables: [0, 1, 2, 3, 4, 5]
Fase 1: True
Number of artificial variables: 2


Fase I
Objective function of fase I: [0, 0, 0, 0, 0, 0, 1, 1]


Iteration: 1

Matrix B:
1	0	
0	1	


Index of basic variables: [6, 7]
Index of non-basic variables: [0, 1, 2, 3, 4, 5]


CB : [1, 1]
XB :3.00 5.00 
X:0.00 0.00 0.00 0.00 0.00 0.00 3.00 5.00 
lambda(shadow price):1.00 1.00 
SN: -1.00 -2.00 -5.00 5.00 1.00 -1.00 
solution is not optimal
Entering: SN3
y: 3.00 2.00 
Leaving: B1
leaver = x7
entrant = x3


Iteration: 2

Matrix B:
3	0	
2	1	


Index of basic variables: [2, 7]
Index of non-basic variables: [0, 1, 6, 3, 4, 5]


CB : [0, 1]
XB :1.00 3.00 
X:0.00 0.00 1.00 0.00 0.00 0.00 0.00 3.00 
lambda(shadow price):-0.67 1.00 
SN: 0.67 -2.00 1.67 0.00 -0.67 -1.00 
solution is not optimal
Entering: SN2
y: 0.00 2.00 
Leaving: B2
leaver = x8
en

In [9]:

'''*********************************************** dual *******************************************************'''


def dual(d_problem_type,d_restrictions,d_objective_function0,d_operators,d_b):
    
    problem_type = d_problem_type
    
    objective_function0 = d_objective_function0
    

    restrictions = d_restrictions

                  
    operators = d_operators
                
    b0= d_b

    ############################## Dual entrants ###############################
    d_operators=[]
    if problem_type=='max':
        d_problem_type='min'
        rest=np.array(restrictions)
        dual_rest=rest.T
        d_restrictions=dual_rest.tolist()
        d_b=objective_function0
        d_objective_function0=b0
        
        for i in range(len(d_b)):
            d_operators.append('>=')
    else:
        d_problem_type='max'
        rest=np.array(restrictions)
        dual_rest=rest.T
        d_restrictions=dual_rest.tolist()
        d_b=objective_function0
        d_objective_function0=b0
        for i in range(len(d_b)):
            d_operators.append('<=')
    
   
    ############################################################################
    obj=[]
    for i in range(len(restrictions[0])):
        obj.append(objective_function0[i])
    ##########################non_unique optimal solution#######################
    
    s=[]
    for i in range(len(obj)):
        s.append(0)
    f=[]
    bb=500000000000
    mat_objective_function0=np.array(objective_function0)
    mat_rest=np.array(restrictions)

    for i in range(len(b0)):
        f.append(mat_objective_function0/mat_rest[i])
    
    for i in range(len(b0)):
        if len(np.unique(f[i]))==1:
            s=restrictions[i]
            bb=b0[i]
            rest_index=i+1
    s=np.array(s)
    s=s.reshape(1,len(obj))
    ############################################################################
    # creating standard form #
    A, objective_function, b, basic_vars, Non_basic_vars, fase1, nVarArtificial = standard_form(problem_type, objective_function0, restrictions, operators, b0)

    problem = False

    if fase1:
        itr = 1
        objective_function_fase1 = [0] * len(objective_function)
        for i in range(len(objective_function_fase1) - 1, len(objective_function_fase1) - 1 - nVarArtificial, -1): # assigns 1 for all costs of artificial variables
            objective_function_fase1[i] = 1

        print('\n\n\033[33mFase I\033[m')
        print('Objective function of fase I: {}'.format(objective_function_fase1))
      

        while True:

            B = update_B(A, basic_vars)
            B_inv=np.linalg.inv(B)
            
            CB=[]
            for i in basic_vars:
                CB.append(objective_function_fase1[i])
            
            printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
            Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))
            mXb=np.array(Xb)
            lbda = step2_1(B, basic_vars, objective_function_fase1)
            #SN = CN - NT* lamda
            vet_cnk = step2_2(objective_function_fase1, lbda, A, Non_basic_vars)
            ##########################################################
            Binvdotrest=np.dot(B_inv, restrictions)
            
            CBdot=np.dot(CB,Binvdotrest)
#             print('CB*B^-1*A:',CBdot)
#             print("\n")
           
            CBXb=np.dot(CB,Xb)
#             print('CB*Xb :',CBXb)
#             print("\n")
            CBBinv=np.dot(CB,B_inv)
#             print('CB*B^-1 :',CBBinv)
#             print("\n")
            BinvA=np.dot(B_inv,restrictions)
#             print('B^-1*A :', BinvA)
#             print("\n")
           
            
            ##
            finish, k = step3(vet_cnk, True)

            if finish:
                break
            # column of the intery variable in table 
            y = step4(B, A, Non_basic_vars, k)
            #finding the exiting variable by considering the ratios(index of B column)
            leaver, problem = step5_fase1(Xb, y)
            
            
        ########################checking degeneracy#######################
         
            igreg=np.array(y)
            igreg=igreg.reshape(len(y),1)
            mXb=mXb.reshape((len(Xb)),1)
            ratio=mXb/igreg
            ratio=np.reshape(ratio,len(y))
            ratio_list=ratio.tolist()
            t1 = min(i for i in ratio_list if i >= 0)
            ratio_list.remove(t1)
            a1 = [x for x in ratio_list if x>=0]
            if len(a1)>0 :
              t2 = min(a1)
              if t1 == t2 :
                print(colored('we have degeneracy!','red'))

        
        
        ##################################################################

            if problem:
                break

            basic_vars, Non_basic_vars, finish = step6_fase1(basic_vars, Non_basic_vars, k, leaver, nVarArtificial)

            if finish:
                break

            itr += 1

        fimFase1(A, nVarArtificial, Non_basic_vars)

    itr = 1
    X = []

    print('\n\n\033[33mFase II\033[m')
    while True:
        

        B = update_B(A, basic_vars)
        
        B_inv=np.linalg.inv(B)
        mB_inv=np.array(B_inv)
        #
        mB_inv=np.around(mB_inv, decimals=5)
        #
        CB=[]
        for i in basic_vars:
            
            CB.append(objective_function[i])
        mCB=np.array(CB)
        printInfo(basic_vars, Non_basic_vars, B, itr,B_inv,objective_function,CB)
        Xb, X = step1(B, b, basic_vars, len(basic_vars) + len(Non_basic_vars))
        

        row_titles=['z']
        for i in basic_vars:
            row_titles.append('x'+str(i+1))

        mrow_titles=np.array(row_titles)
        reshaped_row_titles=mrow_titles.reshape(len(row_titles),1)

        lbda = step2_1(B, basic_vars, objective_function)
        vet_cnk = step2_2(objective_function, lbda, A, Non_basic_vars)
        
        
        ############################################################
        len_mXb=len(Xb)
        mXb=np.array(Xb)
        #
        mXb=np.around(mXb, decimals=5)
        #

        Binvdotrest=np.dot(B_inv, restrictions)
        mBinvdotrest=np.array(Binvdotrest)
 
        CBBinvA=np.dot(CB,Binvdotrest)
        mCBBinvA=np.array(CBBinvA)
#         print('CB*B^-1*A:',CBBinvA)
#         print("\n")
        if problem_type == 'min':
            CBBinvA_c=CBBinvA-obj
        else:
            CBBinvA_c=CBBinvA+obj
        mCBBinvA_c=np.array(CBBinvA_c)
        #
        mCBBinvA_c=np.around(mCBBinvA_c, decimals=5)
        #
#         print('(CB*B^-1*A)-c :',mCBBinvA_c)
#         print("\n")
        
        CBXb=np.dot(CB,Xb)
        mCBXb=np.array(CBXb)
        #
        mCBXb=np.around(mCBXb, decimals=5)
        #
        
#         print('CB*Xb :',CBXb
        CBBinv=np.dot(CB,B_inv)
        mCBBinv=np.array(CBBinv)
        #
        mCBBinv=np.around(mCBBinv, decimals=5)
        #
#         print('CB*B^-1 :',CBBinv)
#         print("\n")
        BinvA=np.dot(B_inv,restrictions)
        mBinvA=np.array(BinvA)
        #
        mBinvA=np.around(mBinvA, decimals=5)
        #
#         print('B^-1*A :', BinvA)
#         print("\n")
        BinvABinv=np.concatenate((mBinvA,mB_inv),1)
   
        mXb=mXb.reshape((len_mXb,1))
        BinvABinvXb=np.concatenate((BinvABinv,mXb),1)
#         print('B^-1*A/B^-1/B^-1*b :',BinvABinvXb)
#         print("\n")

        CBBinvA_cCBBinv=np.concatenate((mCBBinvA_c,mCBBinv),None)

        row0=np.concatenate((CBBinvA_cCBBinv, mCBXb),None)
       
#         print('CB*B^-1*A-c/CB*B^-1/CB*B^-1*b:', row0)
#         print("\n")
        len_row0=len(row0.tolist())
        
        table=np.concatenate((row0.reshape((1,len_row0)),BinvABinvXb),0)
        table=np.concatenate((reshaped_row_titles,table),1)
       ############################################################################ 

        #################### fixing table for '=' operators###############
        
        
        delete=[]
        for i in range(len(operators)):
            if operators[i] == '=':
                delete.append(i)

        a=0    
        delete=sorted(delete)
    #if 
        for i in range(len(delete)):

            table = np.delete(table,len(obj)+delete[i]+a+1, 1)
            a += -1
            
        
        ############################## table #################################
        p,u=np.shape(table)
        #print(p,u)
        x = PrettyTable()
        column_num=['_']
        for j in range(u-1):
            column_num.append('X'+str(j+1))
        column_num[j+1]='RHS'
        for i in range(p):
            x.add_row(table[i,:])

        x.field_names = column_num
        print(x)
        ####################################################################
                              
        finish, k = step3(vet_cnk, False)

        if finish:
            break

        y = step4(B, A, Non_basic_vars, k)
        leaver, problem = step5(Xb, y)
        
        ########################checking degeneracy#######################
        
        igreg2=np.array(y)
        igreg2=igreg2.reshape(len(y),1)
        mXb=mXb.reshape((len(Xb)),1)
        ratio2=mXb/igreg2
        ratio2=np.reshape(ratio2,len(y))
        ratio_list2=ratio2.tolist()
        t12 = min(i for i in ratio_list2 if i >= 0)
        ratio_list2.remove(t12)
        a12 = [x for x in ratio_list if x>=0]
        if len(a12)>0 :
          t22 = min(a12)
          if t12 == t22 :
            print(colored('we have degeneracy!','red'))

        ##################################################################
        

        if problem:
            break

        basic_vars, Non_basic_vars = step6(basic_vars, Non_basic_vars, k, leaver)
        
        itr += 1

    if not problem:
        e=[]
        for i in range(len(restrictions[0])):
            e.append(X[i])
        e=np.array(e)
        e=e.reshape(len(obj),1)
        dot=np.dot(s,e)

        print('\033[33mFinal dual Table\033[m')
        
        final(restrictions, objective_function, problem_type, X)
        
        if dot[0]==bb:
            print(colored('we have none_unique optimal solution !','red'))
            print(colored('All the points on restriction'+str(rest_index)+' are optimal solutions.' ,'red'))
            print("\n\n")   
        ################## sensitivity analysis 1 ####################
        print("\n\n")
        import sympy as sym
        from sympy.solvers.inequalities import reduce_rational_inequalities
        for j in range(len(b0)):
            cc = sym.Symbol('b'+str(j)) 
            testb=deepcopy(b0)
            testb[j]=cc
            testb=np.array(testb)
            testb=testb.reshape(len(b0),1)
            
            BinvXb=np.dot(mB_inv,testb)
            BinvXb=np.reshape(BinvXb,len(b0))
            
            print(colored('sensitivity analysis b'+str(j)+' :','green'))
            for i in range(len(b0)):
            
                print(reduce_rational_inequalities([[BinvXb[i] > 0]], cc))

        ################## sensitivity analysis 2 #######################
#         print("\n\n")
 
#         for f in range(len(obj)):
#             bv=deepcopy(basic_vars)
#             bv=np.array(bv)
#             AA=np.array(restrictions)
#             AA=AA.reshape(len(b0),len(obj))
#             uu=sym.Symbol('c'+str(f))
#             ob=deepcopy(obj)
#             ob[f]=uu
#             result=[]
#             Cb=deepcopy(CB)
#             result=np.where(bv == f)
#             result=np.array(result)
#             Cb[result[0][0]]= uu
# #             print('kkkkkkkkk',result,result[0][0],Cb,ob)
# #             print('jjjjjjjjjjjjjjj',mB_inv)
#             newCB=np.array(Cb)
#             newCB=newCB.reshape(1,len(Cb))
#             CBBinverse=np.dot(newCB,mB_inv) #
#             CBBinverseA=np.dot(CBBinverse,AA)
#             CBBinverse=np.reshape(CBBinverse,len(CB))
#             CBBinverseA_c=CBBinverseA-ob #
#             CBBinverseA_c=np.reshape(CBBinverseA_c,len(obj))
#             print(colored('sensitivity analysis c'+str(f)+' :','blue'))
            #print('ffffffffffffffff',CBBinverseA_c,CBBinverse)
#             for j in range(len(CB)):
#                 print(reduce_rational_inequalities([[ CBBinverse[j] < 0]], uu ))
#             for h in range(len(obj)):
#                 print(reduce_rational_inequalities([[ CBBinverseA_c[h] < 0]], uu ))
                


    return
dual(d_problem_type,d_restrictions,d_objective_function0,d_operators,d_b)




Matrix A:
1	0	1	0	0	0	1	0	0	0	
0	-2	0	1	0	0	0	1	0	0	
3	-2	0	0	1	0	0	0	1	0	
-3	2	0	0	0	1	0	0	0	1	


Objective function: [-3, 5, 0, 0, 0, 0, 0, 0, 0, 0]
Vector b: [5, 12, 7, 18]
Index of Basic variables: [6, 7, 8, 9]
Index of Non-basic variables: [0, 1, 2, 3, 4, 5]
Fase 1: True
Number of artificial variables: 4


Fase I
Objective function of fase I: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


Iteration: 1

Matrix B:
1	0	0	0	
0	1	0	0	
0	0	1	0	
0	0	0	1	


Index of basic variables: [6, 7, 8, 9]
Index of non-basic variables: [0, 1, 2, 3, 4, 5]


CB : [1, 1, 1, 1]
XB :5.00 12.00 7.00 18.00 
X:0.00 0.00 0.00 0.00 0.00 0.00 5.00 12.00 7.00 18.00 
lambda(shadow price):1.00 1.00 1.00 1.00 
SN: -1.00 2.00 -1.00 -1.00 -1.00 -1.00 
solution is not optimal
Entering: SN1
y: 1.00 0.00 3.00 -3.00 
Leaving: B3
leaver = x9
entrant = x1


Iteration: 2

Matrix B:
1	0	1	0	
0	1	0	0	
0	0	3	0	
0	0	-3	1	


Index of basic variables: [6, 7, 0, 9]
Index of non-basic variables: [8, 1, 2, 3, 4, 5]


CB : [1, 1, 0, 1]
XB :2.67 

/home/shiny/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in true_divide
/home/shiny/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:131: RuntimeWarning: divide by zero encountered in true_divide
/home/shiny/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:302: RuntimeWarning: divide by zero encountered in true_divide
